In [1]:
# import libraries

import os
from pathlib import Path
import sys
import time

import pandas as pd
import numpy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt


import folium
import geopandas as gpd
import fiona
from shapely.geometry import Point, LineString
import pyproj

import json

%matplotlib inline


print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


In [2]:
# 버스, 지하철 등 대중교통 관련 데이터 정리

bus = pd.read_csv('data/7.부산시남구_버스정류소정보.csv')
bus.head(5)

,stn_no,stn_id,stn_nm,lon,lat,stn_info
0,7002.0,180710101,동부변전소,129.094314,35.147485,일반
1,7003.0,180720201,성모병원대남초등학교입구,129.095258,35.147608,일반
2,7004.0,180790101,낙농마을,129.098048,35.145072,일반
3,7005.0,199490101,대남교차로,129.104207,35.140628,일반
4,7006.0,199480301,경성대 부경대역,129.102307,35.138561,일반


In [3]:
metro = pd.read_csv('data/8.부산시남구_지하철역정보.csv')
new_metro = metro.drop(columns=['stn_ex_ln', 'stn_ex_info']) # 칼럼 제거 -> 환승역 관련 해당사항 없음
new_metro

,stn_nm,stn_ln_no,stn_ln_info,stn_ex_yn,lon,lat
0,경성대부경대역,S2602,부산도시철도 2호선,일반역,129.100548,35.137585
1,대연역,S2602,부산도시철도 2호선,일반역,129.092161,35.135153
2,못골역,S2602,부산도시철도 2호선,일반역,129.084415,35.134731
3,지게골역,S2602,부산도시철도 2호선,일반역,129.074365,35.135574
4,문현역,S2602,부산도시철도 2호선,일반역,129.067399,35.139151
5,국제금융센터부산은행역,S2602,부산도시철도 2호선,일반역,129.066755,35.145817


In [4]:
# 지도로 나타내기

'''
bus_json = bus.to_json()
metro_json = metro.to_json()
#bus.to_crs(epsg='4326')
#metro.to_crs(epsg='4326')

loc = [35.1300,129.100]

map_pt = folium.Map(location=loc, tiles = 'cartodbpositron', zoom_start=13)

for i in range(len(bus)):
    folium.Marker(list(bus.iloc[i][['lat', 'lon']]),
                  popup='버스정류장',
                  icon=folium.Icon(color='green',  icon='fas fa-bus', prefix='fa')).add_to(map_pt) # fontawesome 활용하려면 prefix도 바꿔야 함
    
for i in range(len(metro)):
    folium.Marker(list(metro.iloc[i][['lat', 'lon']]),
                  popup='지하철역',
                  icon=folium.Icon(color='red', icon='subway', prefix='fa')).add_to(map_pt)
        
map_pt
'''

"\nbus_json = bus.to_json()\nmetro_json = metro.to_json()\n#bus.to_crs(epsg='4326')\n#metro.to_crs(epsg='4326')\n\nloc = [35.1300,129.100]\n\nmap_pt = folium.Map(location=loc, tiles = 'cartodbpositron', zoom_start=13)\n\nfor i in range(len(bus)):\n    folium.Marker(list(bus.iloc[i][['lat', 'lon']]),\n                  popup='버스정류장',\n                  icon=folium.Icon(color='green',  icon='fas fa-bus', prefix='fa')).add_to(map_pt) # fontawesome 활용하려면 prefix도 바꿔야 함\n    \nfor i in range(len(metro)):\n    folium.Marker(list(metro.iloc[i][['lat', 'lon']]),\n                  popup='지하철역',\n                  icon=folium.Icon(color='red', icon='subway', prefix='fa')).add_to(map_pt)\n        \nmap_pt\n"

In [5]:
# 행정경계 데이터 및 운전가능 인구 데이터와 연계

population = pd.read_csv('data/부산시남구_인구(2021-04기준).csv', sep=',', encoding='utf-8')
emd = gpd.read_file('data/14.부산시남구_행정경계(읍면동).geojson')

emd = emd.astype({'ADM_DR_CD': int}) # population의 cd와 데이터 타입이 동일해야 choropleth로 나타낼 때 mapping됨

In [6]:
# crs값 조정 및 위치 파라미터 지정

emd.to_crs(epsg='4326')
converted_emd = emd.to_json()

bus_json = bus.to_json()
metro_json = metro.to_json()

loc = [35.1300,129.100]

map_popnpt = folium.Map(location=loc, tiles='cartodbpositron',zoom_start=13)

# 운전가능 인구/행정경계 지도로 나타내기

choropleth = folium.Choropleth(
              geo_data=converted_emd, 
              data=population,
              columns=['code','pop'],
              key_on='feature.properties.ADM_DR_CD',
              fill_color='PuRd',
              fill_opacity = 0.3,
              line_weight = 0.3, 
              bins = 4,
              highlight=True,
              legend_name='부산 남구 동별 인구 (명)')

# popup 추가하기
choropleth.geojson.add_child(folium.features.GeoJsonTooltip(['ADM_DR_NM'], labels=False))

choropleth.add_to(map_popnpt)

# 버스정류장/지하철역 지도로 나타내기
for i in range(len(bus)):
    folium.Marker(list(bus.iloc[i][['lat', 'lon']]),
                  popup='버스정류장',
                  icon=folium.Icon(color='green',  icon='fas fa-bus', prefix='fa')
                 ).add_to(map_popnpt)
    
for i in range(len(metro)):
    folium.Marker(list(metro.iloc[i][['lat', 'lon']]),
                  popup='지하철역',
                  icon=folium.Icon(color='red', icon='subway', prefix='fa')
                 ).add_to(map_popnpt)
        
map_popnpt